In [366]:
import numpy as np
import pandas as pd
import random

In [367]:
class GeneticAlgorithm:
    """ M ------- size of population, must be even
        N ------- number of genes in a chromosome
        MaxGen -- maximum number of generations to run for
        Pc ------ probability of crossover
        pm ------ probability of mutation
        obj ----- name of fitness function
        ub ------ vector of upper bounds for the genes
        lb ------ vector of lower bounds for the genes
    """

    def __init__(self, m, n, maxgen, pc, pm, er, ub, lb, fitness_function_object, chromosome_to_include=[],
                 mutate_percent=False, mutate_percentage=.01, pop_to_include=[], view_gen=False):
        self.cgcurve = np.zeros(maxgen)
        self.m = m
        self.n = n
        self.maxgen = maxgen
        self.pc = pc
        self.pm = pm
        self.mutate_percentage = mutate_percentage
        self.mutate_high = 1 + mutate_percentage
        self.mutate_low = 1 - mutate_percentage
        self.er = er
        self.ub = ub
        self.lb = lb
        self.view_gen = view_gen
        self.chromosome_to_include = chromosome_to_include
        if pop_to_include == []:
            self.population = self.initialization()
        else:
            self.population = pop_to_include
        self.fitness_function = fitness_function_object
        self.population = self.fitness_function(self.population)
        self.mutate_percent = mutate_percent

    def initialization(self):
        # Last 'gene' is the fitness value
        population = np.random.rand(self.m, self.n + 1)
        population[:, :-1] = (self.ub - self.lb) * population[:, :-1] + self.lb
        
        # n_p1 = self.n + 1
        # upper_bound = self.ub[0] + 1
        # population = np.random.randint(5, size=(self.m, 410))
        # population[:, :-1] = (self.ub - self.lb) * population[:, :-1] + self.lb
        # that last column make it 0 for the starting fitness value
        population[:, -1] = np.zeros((1, self.m))
        if self.chromosome_to_include:
            population[0] = self.chromosome_to_include
        return population

    def mutate(self, child):
        # An np array full of mutations
        mutates = (self.ub - self.lb) * np.random.rand(1, self.n) + self.lb
        # HAlf of the time we mutate higher, half the time we mutate lower
        if self.mutate_percent:
            R = np.random.rand()
            if R < .5:
                mutates = self.mutate_low * child[:-1]
            else:
                mutates = self.mutate_high * child[:-1]

        # Create array of 1s and 0s for true/ false within prob mutation
        check_pm = (np.random.rand(1, self.n) < self.pm).astype(int)

        # No need to mutate fitness value
        mutates = np.append(mutates, [0])
        check_pm = np.append(check_pm, [0])
        child = check_pm * mutates + (1 - check_pm) * child

        return child
    
    def mutate_integer(self, child):
        # An np array full of mutations
        mutates = np.random.randint(self.ub[0]+1, size=(1, self.N)) #random.randint(self.u)(self.ub - self.lb) * np.random.rand(1, self.n) + self.lb
        
        # Create array of 1s and 0s for true/ false within prob mutation
        check_pm = (np.random.rand(1, self.n) < self.pm).astype(int)

        # No need to mutate fitness value
        mutates = np.append(mutates, [0])
        check_pm = np.append(check_pm, [0])
        child = check_pm * mutates + (1 - check_pm) * child

        return child

    def elitism(self, population, new_population):
        # number of elite chosen here
        elite_no = round(self.m * self.er)
        sorted_population = population[population[:, -1].argsort()]
        # Keep best from old population
        new_population[-elite_no:, :] = sorted_population[-elite_no:, :]
        return new_population

    def single_cross(self, parent1, parent2):
        # Single point crossover Using vectors
        gene_no = self.n + 1
        cros_pt = random.randint(1, gene_no - 2)
        # if greater than the probability of crossover, it remains the original
        # parent
        r1 = random.uniform(0, 1)
        if r1 >= self.pc:
            child1 = parent1
        else:
            child1 = np.append(parent1[0: cros_pt], parent2[cros_pt:gene_no])

        r2 = random.uniform(0, 1)
        if r2 >= self.pc:
            child2 = parent2
        else:
            child2 = np.append(parent2[0: cros_pt], parent1[cros_pt:gene_no])

        return child1, child2

    def selection(self, population):
        # Sort by fitness values
        sorted_population = population[population[:, -1].argsort()[::-1]]
        # fitness scaling
        sum_fitness_values = np.sum(population[:, -1])
        norm_sorted_population = np.copy(sorted_population)
        norm_sorted_population[:, -1] /= sum_fitness_values

        # Get cumulative sum of fitness values
        chrom_idexs = np.arange(0, self.m, 1)
        # initialize cumulative fitness values vector
        cumsum = np.zeros(self.m)
        for i in chrom_idexs:
            for j in np.arange(i, self.m, 1):
                cumsum[i] += norm_sorted_population[j][-1]

        # Use random number to pick parent number 1
        r = random.uniform(0, 1)
        parent1_idx = self.m - 1
        for i in chrom_idexs:
            if r > cumsum[i]:
                parent1_idx = i - 1
                break

        parent2_idx = parent1_idx
        # to break the while loop in rare cases where we keep getting the same index
        while_loop_stop = 0
        while parent2_idx == parent1_idx:
            while_loop_stop += 1
            r = random.uniform(0, 1)
            if while_loop_stop > 20:
                break
            for i in chrom_idexs:
                if r > cumsum[i]:
                    parent2_idx = i - 1
                    break

        parent1 = sorted_population[parent1_idx]
        parent2 = sorted_population[parent2_idx]

        return parent1, parent2

    
    def run(self):
        if self.view_gen:
            print('Generation # ', 0)
        self.cgcurve[0] = np.max(self.population[:, -1])
        population = self.population

        # Main loop
        for g in np.arange(1, self.maxgen, 1):
            if self.view_gen:
                print()
                print('Generation # ', g)

            # Calculate the fitness values
            population = self.fitness_function(population)

            # Empty array for next population
            new_population = np.zeros((self.m, self.n + 1))
            for k in np.arange(0, self.m, 2):
                # Selection
                [parent1, parent2] = self.selection(population)

                # Crossover
                [child1, child2] = self.single_cross(parent1, parent2)

                # Mutation
                child1 = self.mutate(child1)
                child2 = self.mutate(child2)

                new_population[k] = child1
                new_population[k + 1] = child2

            new_population = self.fitness_function(new_population)

            # Elitism
            population = self.elitism(population, new_population)

            self.cgcurve[g] = np.max(population[:, -1])

        sort_final_pop = population[population[:, -1].argsort()[::-1]]
        best_chrom = sort_final_pop[0]

        return best_chrom, population, self.cgcurve

In [368]:
class DynamicFuzzySystem:
    def __init__(self,
                 output_membership_functions,
                 predictors):
        # create empty data frame for the membership values that stay fixed because we fix the mfs
        self.membership_value_pd = pd.DataFrame(columns=predictors)
        self.membership_func_names_pd = pd.DataFrame(columns=predictors)
        self.output_membership_functions = output_membership_functions

    @staticmethod
    def defuzz_center_of_sums(y, output_shape):
        """ Defuzzification using the center of area
            CONFIRMED, see: https://codecrucks.com/defuzzification-methods-solved-examples/"""
        left = output_shape[0]
        center = output_shape[1]
        right = output_shape[2]
        area = .5 * (right - left)
        # scale the output membership function using rule weight
        scaled_function = center * y * area
        return scaled_function, area

    @staticmethod
    def memb_tri(x, triangle):
        """This function determines the membership value belonging to the user defined triangle
        INPUTS
        x - the value
        triangle - user defined triangle in order:left, center, right"""
        left = triangle[0]
        center = triangle[1]
        right = triangle[2]

        if left <= x < center:
            # Left side of triangle
            membership = (x - left) / (center - left)
        elif center <= x <= right:
            # Right side of triangle
            if (center - right) == 0:
                membership = 1
            else:
                membership = (x - right) / (center - right)
        else:
            # not in triangle
            membership = 0
        return membership

    def input_to_membership(self, input_name, input_column, membership_functions, membership_function_labels):
        """
        Calculates the membership values for all membership functions for all data points for a certain input

        This function takes the parameters:
        input_name - name of input as listed in data
        input_column - the values of each data point for this input
        membership_functions - list of lists the holds the membership function info [[left, center, right], ...]
        membership_function_labels - labels for each of the membership functions
        """
        all_input_data = []
        for input_value in input_column:
            memberships = []
            for membership_function in membership_functions:
                membership = self.memb_tri(input_value, membership_function)
                memberships.append(membership)
            all_input_data.append([memberships])
        self.membership_value_pd[input_name] = pd.DataFrame(all_input_data)
        self.membership_func_names_pd[input_name] = [membership_function_labels]
        print('Membership values calculated for input: ' + input_name)

        
    def input_to_membership_intermediate_level(self, input_value, membership_functions):
        """
        Calculates the membership values for all membership functions for all data points for a certain input

        This function takes the parameters:
        input_value - the value of an input
        membership_functions - list of lists the holds the membership function info [[left, center, right], ...]
        """
        memberships = []
        for membership_function in membership_functions:
            membership = self.memb_tri(input_value, membership_function)
            memberships.append(membership)

        return memberships
        
        
    @staticmethod
    def check_if_rule_fires(memberships):
        # If each membership value in a rule is 0, then those membership functions do not fire together for this data
        # point
        memberships = np.array(memberships)
        is_all_zero = np.all((memberships == 0))
        return is_all_zero
    

    def rules2_1(self, input_1_mfs, input_2_mfs):
        """
        Calculate and rules for a two input one output FIS for one data point

        data_point - index of the current data point being calculated for
        """
        and_rules = []
        rule_number = 0
        non_fire_rules = []
        for mem_1 in input_1_mfs:
            for mem_2 in input_2_mfs:
                and_rules.append(min(mem_1, mem_2))
                if self.check_if_rule_fires([mem_1, mem_2]):
                    non_fire_rules.append(rule_number)
                    
                rule_number += 1
        return and_rules, non_fire_rules



    def rules3_1(self, input_1_mfs, input_2_mfs, input_3_mfs):
        """
        Calculate and rules for a three input one output FIS for one data point

        """
        and_rules = []
        rule_number = 0
        non_fire_rules = []
        for mem_1 in input_1_mfs:
            for mem_2 in input_2_mfs:
                for mem_3 in input_3_mfs:
                    and_rules.append(min(mem_1, mem_2, mem_3))
                    if self.check_if_rule_fires([mem_1, mem_2, mem_3]):
                        non_fire_rules.append(rule_number)
                        
                    rule_number += 1
        return and_rules, non_fire_rules
    
    
    def rules4_1(self, input_1_mfs, input_2_mfs, input_3_mfs, input_4_mfs):
        """
        Calculate and rules for a four input one output FIS for one data point

        data_point - index of the current data point being calculated for
        """
        and_rules = []
        rule_number = 0
        non_fire_rules = []
        for mem_1 in input_1_mfs:
            for mem_2 in input_2_mfs:
                for mem_3 in input_3_mfs:
                    for mem_4 in input_4_mfs:
                        and_rules.append(min(mem_1, mem_2, mem_3, mem_4))
                        if self.check_if_rule_fires([mem_1, mem_2, mem_3, mem_4]):
                            non_fire_rules.append(rule_number)
                        rule_number += 1
                        
        return and_rules, non_fire_rules


    def defuzzification(self, and_rules, map_to_output_func, alternate_output_functions=[]):
        """
        defuzzification of the system to get final crisp output

        and_rules - result of the and rules
        map_to_output_func - GA determines which output function this rule maps to

        """
        output_membership_funcs = self.output_membership_functions
        if alternate_output_functions:
            output_membership_funcs = alternate_output_functions
        lam_numerator = 0
        lam_denominator = 0
        for and_rule, map_to_output_func in zip(and_rules, map_to_output_func):
            # Defuzzification using the center of sums
            if map_to_output_func > (len(output_membership_funcs)-1):
                map_to_output_func = len(output_membership_funcs)-1
            if and_rule != 0:
                output_value = self.defuzz_center_of_sums(and_rule, output_membership_funcs[int(map_to_output_func)])
                lam_numerator += output_value[0]
                lam_denominator += output_value[1]

        # Check to make sure denominator is not 0
        if lam_denominator == 0:
            lam = 0
        else:
            lam = lam_numerator / lam_denominator

        # return the output of this fis
        return lam

In [369]:
# Predictors after coversions
predictors_final = ["GSS_Type_numeric",
                  "GSS_Propulsion_numeric",
                  "GSS_Main.engines..Model",
                  "GSS_Main.engines..Designer",
                  "GSS_Main.engines..Builder.code",
                  "GSS_Gross.tonnage",
                  "GSS_Deadweight",
                  "GSS_TEU",
                  "GSS_Insulated.capacity",
                  "GSS_Length.overall",
                  "GSS_Length.between.perpendiculars",
                  "GSS_Service.speed",
                  "GSS_Main.engines..Number.of.main.engines",
                  "GSS_Main.engines..Max..power",
                  "age_in_months"]

In [370]:
import pickle

# open a file, where you stored the pickled data
file = open('data_folds_7_11', 'rb')

# dump information to that file
df = pickle.load(file)

# close the file
file.close()

In [371]:
current_fold = 0
train_idxes = df['folds'][current_fold][0]
test_idxes = df['folds'][current_fold][1]

train_dataset_X = df['X'].loc[list(train_idxes), :]
train_dataset_Y = df['Y'].loc[list(train_idxes)]

test_dataset_X = df['X'].loc[list(test_idxes), :]
test_dataset_Y = df['Y'].loc[list(test_idxes)]
# train_dataset_X[]

In [372]:
# Mfs for output and fixed intermediate layers
mfs_fixed = [[0, 0, .4], [.2, .5, .8], [.6, 1, 1]]
mfs_fixed_3_1 = [[0, 0, .3], [.2, .35, .55], [.45, .65, .8], [.7, 1, 1]]


In [373]:
# Define the Input Membership Functions
# GSS_Type_numeric
mfs_predictor0 = [[-.1, 0, .1], [0.9, 1, 1.1], [1.9, 2, 2.1], [2.9, 3, 3.1], [3.9, 4, 4.1], [4.9, 5, 5.1], [5.9, 6, 6.1], [6.9, 7, 7.1], [7.9, 8, 8.1]]
membership_function_labels0 = ['type 0', 'type 1', 'type 2', 'type 3', 'type 4', 'type 5', 'type 6', 'type 7', 'type 8']

# "GSS_Propulsion_numeric"
mfs_predictor1 = [[-.1, 0, .1], [0.9, 1, 1.1], [1.9, 2, 2.1], [2.9, 3, 3.1]]
membership_function_labels1 = ['type 0', 'type 1', 'type 2', 'type 3']

# "GSS_Main.engines..Model_numeric"
mfs_predictor2 = [[0, 150, 250], [170, 233, 400], [300, 400, 670]]
membership_function_labels2 = ['young parts', 'middle age parts', 'old parts']

# "GSS_Main.engines..Designer_numeric",
mfs_predictor3 = [[130, 350, 375], [365, 420, 455], [445, 465, 505], [500, 545, 655]]
membership_function_labels3 = ['young parts', 'middle young parts', 'middle old parts', 'old parts']

# "GSS_Main.engines..Builder.code_numeric"
mfs_predictor4 = [[80, 220, 300], [280, 335, 400], [390, 440, 655]]
membership_function_labels4 = ['young parts', 'middle age parts', 'old parts']
    
# "GSS_Gross.tonnage"
mfs_predictor5 = [[0, 0, 50000], [45000, 60000, 125000], [120000, 140000, 175000]]
membership_function_labels5 = ['small', 'medium', 'large']

# "GSS_Deadweight"
mfs_predictor6 = [[0, 0, 50000], [45000, 60000, 125000], [120000, 170000, 330000]]
membership_function_labels6 = ['small', 'medium', 'large']
    
# "GSS_TEU"
mfs_predictor7 = [[-1.1, -1, -.9], [0, 0, 2500], [2200, 4000, 7600], [7400, 13000, 18000]]
membership_function_labels7 = ['no data', 'small', 'medium', 'large']
    
# "GSS_Insulated.capacity"
mfs_predictor8 = [[-1.1, -1, -.9], [0, 0, 1000], [800, 5000, 10000], [9000, 15000, 22000]]
membership_function_labels8 = ['no data', 'small', 'medium', 'large']
    
# "GSS_Length.overall"
mfs_predictor9 = [[40, 80, 150], [140, 180, 230], [220, 280, 400]]
membership_function_labels9 = ['small', 'medium', 'large']

# "GSS_Length.between.perpendiculars"
mfs_predictor10 = [[40, 75, 150], [130, 165, 230], [220, 280, 380]]
membership_function_labels10 = ['small', 'medium', 'large']
    
# "GSS_Service.speed"
mfs_predictor11 = [[-1.1, -1, -.9], [0, 10, 12], [11, 13.5, 20], [18, 24, 28]]
membership_function_labels11 = ['no data', 'small', 'medium', 'large']

# "GSS_Main.engines..Number.of.main.engines"
mfs_predictor12 = [[0, 1, 1.8], [1.5, 3, 6]]
membership_function_labels12 = ['small', 'large']

# "GSS_Main.engines..Max..power"
mfs_predictor13 = [[0, 0, 20000], [18000, 40000, 60000], [50000, 68000, 81000]]
membership_function_labels13 = ['small', 'medium', 'large']

# "age_in_months"
mfs_predictor14 = [[40, 130, 200], [180, 210, 380], [360, 400, 670]]
membership_function_labels14 = ['young', 'middle age', 'old']

all_membership_functions = [mfs_predictor0, mfs_predictor1, mfs_predictor2, mfs_predictor3, mfs_predictor4,
                            mfs_predictor5, mfs_predictor6, mfs_predictor7, mfs_predictor8, mfs_predictor9,
                            mfs_predictor10, mfs_predictor11, mfs_predictor12, mfs_predictor13, mfs_predictor14]

all_membership_function_labels = [membership_function_labels0, membership_function_labels1, membership_function_labels2, membership_function_labels3, membership_function_labels4,
                                  membership_function_labels5, membership_function_labels6, membership_function_labels7, membership_function_labels8, membership_function_labels9,
                                  membership_function_labels10, membership_function_labels11, membership_function_labels12, membership_function_labels13, membership_function_labels14]

In [374]:
fis_class = DynamicFuzzySystem(mfs_fixed, predictors_final)

In [375]:
# Calcualte the membership values for the inputs - these will stay fixed because the membership functions stay fixed
for predictor_input, mfs_predictor, membership_function_labels in zip(predictors_final, all_membership_functions, all_membership_function_labels):
    fis_class.input_to_membership(predictor_input, train_dataset_X[predictor_input], mfs_predictor, membership_function_labels)

Membership values calculated for input: GSS_Type_numeric
Membership values calculated for input: GSS_Propulsion_numeric
Membership values calculated for input: GSS_Main.engines..Model
Membership values calculated for input: GSS_Main.engines..Designer
Membership values calculated for input: GSS_Main.engines..Builder.code
Membership values calculated for input: GSS_Gross.tonnage
Membership values calculated for input: GSS_Deadweight
Membership values calculated for input: GSS_TEU
Membership values calculated for input: GSS_Insulated.capacity
Membership values calculated for input: GSS_Length.overall
Membership values calculated for input: GSS_Length.between.perpendiculars
Membership values calculated for input: GSS_Service.speed
Membership values calculated for input: GSS_Main.engines..Number.of.main.engines
Membership values calculated for input: GSS_Main.engines..Max..power
Membership values calculated for input: age_in_months


In [376]:
# # Rule and aggregation calculations for fixed inputs - these will stay fixed so calculation occurs once here
# and_rules_values_pd = pd.DataFrame(columns=["propulsion_power", "propulsion_age", "weight", "carrying_capacity", "length"])
# non_fire_rules_values_pd = pd.DataFrame(columns=["propulsion_power", "propulsion_age", "weight", "carrying_capacity", "length"])

# # the indexes in the membership_value_pd are corresponding to the length of the trianing dataset
# # the indexes when accesing from the actual dataset are corresponding to which values were chosen in the 
# for data_point in np.arange(0, len(train_dataset_X)):
#     propulsion_power_and_rules, pp_non_fire = fis_class.rules4_1(fis_class.membership_value_pd["GSS_Propulsion_numeric"][data_point],
#                                                                  fis_class.membership_value_pd["GSS_Main.engines..Number.of.main.engines"][data_point],
#                                                                  fis_class.membership_value_pd["GSS_Main.engines..Max..power"][data_point],
#                                                                  fis_class.membership_value_pd["GSS_Service.speed"][data_point])

#     propulsion_age_and_rules, pa_non_fire = fis_class.rules3_1(fis_class.membership_value_pd["GSS_Main.engines..Model"][data_point],
#                                                                fis_class.membership_value_pd["GSS_Main.engines..Designer"][data_point],
#                                                                fis_class.membership_value_pd["GSS_Main.engines..Builder.code"][data_point])

#     weight_and_rules, w_non_fire = fis_class.rules2_1(fis_class.membership_value_pd["GSS_Gross.tonnage"][data_point],
#                                                       fis_class.membership_value_pd["GSS_Deadweight"][data_point])

#     carrying_capacity_and_rules, cc_non_fire = fis_class.rules2_1(fis_class.membership_value_pd["GSS_TEU"][data_point],
#                                                                   fis_class.membership_value_pd["GSS_Insulated.capacity"][data_point])

#     length_and_rules, l_non_fire = fis_class.rules2_1(fis_class.membership_value_pd["GSS_Length.overall"][data_point],
#                                                       fis_class.membership_value_pd["GSS_Length.between.perpendiculars"][data_point])
    
#     and_rules_values_pd.loc[datapoint] = [propulsion_power_and_rules, propulsion_age_and_rules, weight_and_rules, carrying_capacity_and_rules, length_and_rules]
#     non_fire_rules_values_pd.loc[datapoint] = [pp_non_fire, pa_non_fire, w_non_fire, cc_non_fire, l_non_fire]


In [377]:
### STRUCTURE CHANGE FROM JULY 12 
# Rule and aggregation calculations for fixed inputs - these will stay fixed so calculation occurs once here
and_rules_values_pd = pd.DataFrame(columns=["propulsion_power", "propulsion_age", "carrying_capacity", "length"])
non_fire_rules_values_pd = pd.DataFrame(columns=["propulsion_power", "propulsion_age", "carrying_capacity", "length"])

# the indexes in the membership_value_pd are corresponding to the length of the trianing dataset
# the indexes when accesing from the actual dataset are corresponding to which values were chosen in the 
for data_point in np.arange(0, len(train_dataset_X)):
    propulsion_power_and_rules, pp_non_fire = fis_class.rules4_1(fis_class.membership_value_pd["GSS_Propulsion_numeric"][data_point],
                                                                 fis_class.membership_value_pd["GSS_Main.engines..Number.of.main.engines"][data_point],
                                                                 fis_class.membership_value_pd["GSS_Main.engines..Max..power"][data_point],
                                                                 fis_class.membership_value_pd["GSS_Service.speed"][data_point])

    propulsion_age_and_rules, pa_non_fire = fis_class.rules3_1(fis_class.membership_value_pd["GSS_Main.engines..Model"][data_point],
                                                               fis_class.membership_value_pd["GSS_Main.engines..Designer"][data_point],
                                                               fis_class.membership_value_pd["GSS_Main.engines..Builder.code"][data_point])

    carrying_capacity_and_rules, cc_non_fire = fis_class.rules3_1(fis_class.membership_value_pd["GSS_Deadweight"][data_point],
                                                                  fis_class.membership_value_pd["GSS_TEU"][data_point],
                                                                  fis_class.membership_value_pd["GSS_Insulated.capacity"][data_point])

    length_and_rules, l_non_fire = fis_class.rules2_1(fis_class.membership_value_pd["GSS_Length.overall"][data_point],
                                                      fis_class.membership_value_pd["GSS_Length.between.perpendiculars"][data_point])
    
    and_rules_values_pd.loc[data_point] = [propulsion_power_and_rules, propulsion_age_and_rules, carrying_capacity_and_rules, length_and_rules]
    non_fire_rules_values_pd.loc[data_point] = [pp_non_fire, pa_non_fire, cc_non_fire, l_non_fire]


In [378]:
def layers_of_fis(data_point, ga_vec):    
    # Output of first layer
    lam_propulsion_power = fis_class.defuzzification(and_rules_values_pd["propulsion_power"][data_point], ga_vec[0:96])
    lam_propulsion_age = fis_class.defuzzification(and_rules_values_pd["propulsion_age"][data_point], ga_vec[96:132])
    lam_carrying_capacity = fis_class.defuzzification(and_rules_values_pd["carrying_capacity"][data_point], ga_vec[132:180])
    lam_length = fis_class.defuzzification(and_rules_values_pd["length"][data_point], ga_vec[180:189])


    # SIZE FIS - intermediate
    fixed_rules_3_1 =   [0.0, 0.0, 1.0,
                         0.0, 1.0, 2.0,
                         1.0, 2.0, 2.0,

                         0.0, 1.0, 2.0,
                         1.0, 2.0, 2.0,
                         2.0, 2.0, 3.0,

                         1.0, 2.0, 2.0,
                         2.0, 2.0, 3.0,
                         2.0, 3.0, 3.0]
    
    # SIZE FIS - intermediate######
    carrying_capacity_mems = fis_class.input_to_membership_intermediate_level(lam_carrying_capacity, mfs_fixed_3_1)
    length_mems = fis_class.input_to_membership_intermediate_level(lam_length, mfs_fixed_3_1)
    size_and_rules, s_non_fire = fis_class.rules3_1(fis_class.membership_value_pd["GSS_Gross.tonnage"][data_point],
                                                    carrying_capacity_mems,
                                                    length_mems)
    # output of intermediate FIS size
    lam_size = fis_class.defuzzification(size_and_rules, fixed_rules_3_1, alternate_output_functions=mfs_fixed_3_1)

    # PROPUSION FIS - intermediate
    fixed_rules_2_1 = [0, 0, 1,
                       0, 1, 2,
                       1, 2, 2]
    propulsion_power_mems = fis_class.input_to_membership_intermediate_level(lam_propulsion_power, mfs_fixed)
    propulsion_age_mems = fis_class.input_to_membership_intermediate_level(lam_propulsion_age, mfs_fixed)
    propulsion_and_rules, p_non_fire = fis_class.rules2_1(propulsion_power_mems, propulsion_age_mems)
    # output of intermediate FIS propulsion
    lam_propulsion = fis_class.defuzzification(propulsion_and_rules, fixed_rules_2_1)


    # Membership calulations for final FIS
    size_mems = fis_class.input_to_membership_intermediate_level(lam_size, mfs_fixed)
    propulsion_mems = fis_class.input_to_membership_intermediate_level(lam_propulsion, mfs_fixed)
    final_and_rules, final_non_fire = fis_class.rules4_1(fis_class.membership_value_pd["GSS_Type_numeric"][data_point],
                                                         fis_class.membership_value_pd["age_in_months"][data_point],
                                                         size_mems, propulsion_mems)
    lam_final = fis_class.defuzzification(final_and_rules, ga_vec[189:432])
    return lam_final


In [379]:
def classify_points(test_data, test_data_y, chromosome, fit_function_fis):
    """
    Returns
    -------
    total error and the
    the confusion matrix for the points

    """
    def round_to_output(value):
        check_1 = abs(1-value)
        check_0 = abs(0-value)
        if check_1 < check_0:
            rounded_value = 1
        else:
            rounded_value = 0
        return rounded_value
    
    predicted_values = []
    # Initialize confusion matrix
    confusion_matrix = np.zeros((2, 2))
    error = 0
    for ordered_data_row, actual_label in zip(np.arange(0, len(test_data_y)), test_data_y):
        predicted_value = fit_function_fis(ordered_data_row, np.floor(chromosome))
        predicted_values.append(predicted_value)

        # Fill in the confusion matrix
        classified_label = round_to_output(predicted_value)
        confusion_matrix[int(actual_label)][int(classified_label)] += 1
        # Get the error for each point
        error += abs(actual_label - predicted_value)

    return error, confusion_matrix, predicted_values

In [380]:
from sklearn.metrics import roc_auc_score

class FitnessFunctionShipBreaking:
    def __init__(self, train_dataset_X, train_dataset_Y, fis_structure, theta_AB=.5):
        ordered_index = np.arange(0, len(train_dataset_X))
        x_df_add_index = train_dataset_X.insert(loc=0, column="indexed_ordered", value=ordered_index, allow_duplicates=True)
        y_df_add_index = pd.DataFrame(train_dataset_Y)
        y_df_add_index["indexed_ordered"]=ordered_index
        
        self.data = x_df_add_index
        self.y= y_df_add_index
        self.fis_structure = fis_structure
        self.theta=theta_AB

    def fitness_function(self, population):
        """ Original method - how close is each point to the actual prediction""" 
        for gene in population:
            error = 0
            for ordered_data_row, actual_value in zip(self.y["indexed_ordered"], self.y["dismantled"]): #.index.values: #, actual_value in zip(self.data, self.y):
                predicted_value = self.fis_structure(ordered_data_row, np.floor(gene))
                current_error = abs(actual_value - predicted_value)
                error += current_error
                if actual_value == 1 and current_error > .5:
                    error+=10 # high penalty for predicting no shipbreaking when there is a shipbreak
            if error == 0:
                gene[-1] = 1
            else:
                gene[-1] = 1 / error
        return population
    
    def fitness_function_AB(self, population):
        """New method - suggested by antonio in july 12 meeting
           Does not take into account the actual prediction closeness to the output it is rounded to. Can we force this to be taken into account somehow"""
        for gene in population:
            error_final = 0
            error, confusion_matrix, predicted_values = classify_points(self.data, self.y["dismantled"], gene, self.fis_structure)
            auc_score = roc_auc_score(self.y["dismantled"], predicted_values)
            if confusion_matrix[1][1]==0:
                my_calc = 0
            else:
                my_calc = confusion_matrix[1][1]/(confusion_matrix[1][1] + confusion_matrix[0][1] + confusion_matrix[1][0]) #7 18
            final_score = self.theta * auc_score + (1-self.theta)*my_calc
            gene[-1] = final_score
        print(final_score) # print a the last final score to see how things are coming    
        return population

In [381]:
# fitness_object = FitnessFunctionShipBreaking(train_dataset_X, train_dataset_Y)
# fitness_object.data.index.values

In [382]:
# rules = []
# for x1 in np.arange(0,3):
#     for x2 in np.arange(0, 3):
#         for x3 in np.arange(0,3):
#             calc = (x1+x2+x3)/2
#             if calc <= 1:
#                 rules.append(np.floor(calc))
#             if calc > 1:
#                 rules.append(np.ceil(calc))
# rules

In [ ]:
from joblib import dump, load
import time

# Number of chromosomes in a population, must be even
M = 40
N = 432
Pc = .90
Pm = .2
Er = .1
theta=.5
# fitness_object = FitnessFunctionShipBreaking(train_dataset_X, train_dataset_Y)
fitness_object = FitnessFunctionShipBreaking(train_dataset_X, train_dataset_Y, layers_of_fis, theta_AB=theta)
# Make sure the cases input and output bounds are adjusted accordingly
Lb = np.zeros([1, N])
Ub = np.ones([1, N]) * 3


# Only run 2500
MaxGen = 200
Percentage_to_mutate = .1
previous_population = []

tic = time.perf_counter()  # t to track the time it takes to train
ga = GeneticAlgorithm(M, N, MaxGen, Pc, Pm, Er, Ub, Lb, fitness_object.fitness_function_AB, chromosome_to_include=[],
                      mutate_percent=True, mutate_percentage=Percentage_to_mutate, pop_to_include=previous_population,
                      view_gen=True)
best_chromosome, population, fitness_over_time = ga.run()
error_over_time = 1 / fitness_over_time
toc = time.perf_counter()
tt1 = (toc - tic)
print("time to run: ", tt1, " sec")


#  Save results from  run
results_df = pd.DataFrame({'Error over time': [error_over_time],
                           'Population' : [population],
                           'Best Chromosome' : [best_chromosome],
                           'Time, MaxGen, M, theta' : [[tt1, MaxGen, M, theta]]})
dump(results_df, 'models/run_200gen40pop_05theta_my_calc.joblib') 
                          

0.2911792966511095
Generation #  0

Generation #  1
0.2911792966511095
0.3342685909485982

Generation #  2
0.4131887755102041
0.19920774475631384

Generation #  3
0.4131887755102041
0.27424698067632847

Generation #  4
0.4183173511753176
0.3540316778211116

Generation #  5
0.3911591614906832

Generation #  6
0.43263864333393465
0.3759955976312919

Generation #  7
0.43263864333393465
0.3536332316297992

Generation #  8
0.43263864333393465
0.4187463768115942

Generation #  9
0.43263864333393465
0.3959048089591568

Generation #  10
0.45477981878784046
0.4047457411645055

Generation #  11
0.45508416661392737
0.3904485215445968

Generation #  12
0.4646904468071641
0.4014472017136086

Generation #  13
0.4657368178765625
0.4121028295376122

Generation #  14
0.4714801063887532
0.39281031543052003

Generation #  15
0.4714801063887532
0.44548371276574933

Generation #  16
0.4714801063887532
0.45424185164801567

Generation #  17
0.48432422640031336
0.4082740330058684

Generation #  18
0.484324226